In [1]:
# Helpful links:
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
# Using NN with regression: https://deeplearning4j.org/logistic-regression
# A Caffe implementation https://deshanadesai.github.io/notes/Finetuning-VGG-For-Regression
# Potential problem we might run into: https://stackoverflow.com/questions/42243114/getting-linear-regression-score-from-transfer-learning

In [2]:
from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
#from keras import backend as K
#K.set_image_dim_ordering('th')
from sklearn.model_selection import train_test_split
import numpy as np

Using TensorFlow backend.


In [3]:
# path to the model weights files.
weights_path = '../datasets/cnn_weights/vgg16_weights.h5'

In [4]:
# build the VGG16 network
input_tensor = Input(shape=(224,224,3))
model = applications.VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)
print('Model loaded.')

Model loaded.


In [5]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
print(model.output_shape[1:])
top_model.add(Flatten(input_shape=(model.output_shape[1:])))


# Output layer
# We do random weight intialization
# Maybe this is why our loss is so bad?
top_model.add(Dense(256, activation='relu', kernel_initializer='glorot_uniform'))
top_model.add(Dense(1, activation='linear', name='output', kernel_initializer='glorot_uniform'
))

(7, 7, 512)


In [6]:
# add the model on top of the convolutional base
#model.add(top_model)
new_model = Model(inputs= model.input, outputs = top_model(model.output))
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in new_model.layers[:15]:
    layer.trainable = False

In [11]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
# SGD
#new_model.compile(loss='mean_squared_error',
#              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
#              metrics=['accuracy'])

# RMSprop
new_model.compile(loss='mean_squared_error',
              optimizer=optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

In [12]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

data_path = "../datasets/bikes_im/"
file = open("../datasets/bikes_filtered.csv")
i = -1
X = np.zeros((100, 224, 224, 3))
Y = np.zeros((100, 1))
for data_point in file:
    i += 1
    index, name, msrp = data_point.split(",")
    img_path = data_path + index + '.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    X[i] = image.img_to_array(img)
    Y[i] = int(msrp)
    # TODO: Change this to use the full dataset
    if i == 99:
        #print(i)
        break
#print(X.shape)
#print(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)
print(X_train.shape)
print(y_train.shape)


(90, 224, 224, 3)
(90, 1)


In [13]:
train_generator = train_datagen.flow(
    x = X_train,
    y = y_train,
    batch_size= 16)


validation_generator = test_datagen.flow(
    x = X_test,
    y = y_test,
    batch_size= 16)

epochs = 50
nb_train_samples = X_train.shape[0]
nb_validation_samples = X_test.shape[0]

# fine-tune the model
new_model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

C:\Users\Richard\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:22: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\Richard\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, validation_data=<keras.pre..., steps_per_epoch=5, validation_steps=10)`


Epoch 1/50
6/5 [====================================] - 154s 26s/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/50
6/5 [====================================] - 155s 26s/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/50
6/5 [====================================] - 145s 24s/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 4/50
4/5 [=======================>......] - ETA: 21s - loss: nan - acc: 0.0000e+00

KeyboardInterrupt: 